In [31]:
import sys
sys.path.append('..')
from db1 import *
import networkx as nx
import ipycytoscape

R = PolynomialRing(QQ, 't')
S = PolynomialRing(ZZ, 't')
t = R.gen()

def is_paving(M):
    n = M.size()
    r = M.rank()
    return (len(M.independent_r_sets(r-1)) == binomial(n, r-1))

def q_kl(k, h):
    return kazhdan_lusztig_inverse_uniform(k, h+1) - kazhdan_lusztig_inverse_uniform(k-1, h)

def kl_inverse_fast(M):
    if M.loops(): return R(0)
    k, n = M.rank(), M.size()
    if k == n or k == 0: return R(1)
    if not M.is_connected():
        ans = R(1)
        CC = M.components()
        for N in CC:
            res = M.delete(M.groundset() - N)
            ans = ans * kl_inverse_fast(res)
        return ans

    if is_paving(M):
        return kl_inverse_paving(M)
    if is_paving(M.dual()):
        return kl_inverse_copaving(M)
    """
    if n <= 8 and M.is_connected():
        for i in range(len(mat[n][k])):
            if mat[n][k][i].is_isomorphic(M):
                return ikl[n][k][i]
    """
    LF = M.lattice_of_flats()
    ans = R(0)
    for F in LF:
        if len(F) != n:
            Res = M.delete(M.groundset() - F)
            Con = M.contract(F)
            chi = characteristic_polynomial(Con)(1/t) * t**(Con.rank())
            PPP = kl_inverse_fast(Res)(t) * (-1)**(Res.rank())
            ans = ans + chi * PPP
    assert (t**k * ans(1/t)).numerator() == -ans(t)
    ans = ans.numerator() * (-1)**(k+1)
    return ans.truncate((k+1)//2)

def kazhdan_lusztig_inverse_uniform(k, n):
    if k == n:
        return R(1)
    d = k
    m = n - d
    ans = 0
    for j in range((d-1)//2 + 1):
        ans = ans + m * (d-2*j)/((m+j) * (m+d-j)) * binomial(d, j) * t**j
    return ans * binomial(m+d, d)

def kl_inverse_paving(M):
    assert is_paving(M)
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.hyperplanes():
        h = len(H)
        if h >= k:
            ans = ans - q_kl(k, h)
    return ans

def kl_inverse_copaving(M):
    assert is_paving(M.dual())
    n = M.size()
    k = M.rank()
    ans = kazhdan_lusztig_inverse_uniform(k, n)
    for H in M.dual().hyperplanes():
        h = len(H)
        if h >= n-k:
            ans = ans - kli_vtilde_dual(n-k, h, n) + kazhdan_lusztig_inverse_uniform(h-n+k+1, h) * kazhdan_lusztig_inverse_uniform(n-h-1, n-h)
    return ans

def kli_vtilde_dual(k, h, n):
    return helper1(n-k, h, n)

def helper1(k, h, n):
    c = n - h
    ans1 = kazhdan_lusztig_inverse_uniform(k, n)
    ans2 = helper2(c, k, n)
    ans3 = kazhdan_lusztig_inverse_uniform(k-c+1, h) * kazhdan_lusztig_inverse_uniform(c-1, c)
    return ans1 - ans2 + ans3

def helper2(c, k, n):
    h = n - c
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * (-1)**(c-1+j) * kazhdan_lusztig_inverse_uniform(c-1, c) * t**(k-c-j+1) * chuly(k-c-j+1, n-c-j)(1/t)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(i+j) * t**(k-i-j) * helper4(c, k, n, i, j)(1/t)
    ans = ans.numerator().truncate((k-1)//2 + 1)
    if ans[0] < 0:
        ans = -ans
    return ans

def helper3(c, k, n):
    ans = 0
    for j in range(k-c+1):
        ans = ans + binomial(n-c, j) * uniformKLpoly(c-1, c) * (-1)**(k-c-j+1) * kazhdan_lusztig_inverse_uniform(k-c-j+1, n-c-j)
    for i in range(c-1):
        for j in range(k-i):
            ans = ans + binomial(c, i) * binomial(n-c, j) * (-1)**(k-i-j) * helper2(c-i, k-i-j, n-i-j)
    return -ans

def helper4(c, k, n, i, j):
    ans = 0
    for l in range(c-i-1):
        ans = ans + (-1)**l * (t-1)**(max(n-i-j-l-1, 0))
    for u in range(n-k-1):
        ans = doit_once(ans)
    return ans

def chuly(a, b):
    ans = (t-1)**b
    for i in range(b-a):
        ans = doit_once(ans)
    return ans

def doit_once(p):
    p = p // t**2
    p = p * t
    p = p - p(1)
    return p

def lorenzo(k, h, n):
    c = n - h
    ans1 = uniformKLpoly(k, n) + uniformKLpoly(k-c+1, h) * uniformKLpoly(c-1, c)
    ans2 = helper3(c, k, n)
    return ans1 - ans2

In [32]:
invkl = kl_inverse_fast

def kl(M):
    return M.lattice_of_flats().kazhdan_lusztig_polynomial()(t)

def parallel_connection(m, n):
    G = graphs.CycleGraph(m + n - 2)
    G.add_edge(0, m-1)
    edge_e = frozenset({(0, m - 1)})
    return G, edge_e

def qhat(M):
    return (-1) ^ M.rank() * invkl(M)

def uniformQhat(r, n):
    return (-1) ^ r * uniformQpoly(r, n)

def sum_prhc(M, S):
    return sum(kl(M.delete(M.groundset() - set(F))) * \
            qhat(M.contract(F)) for F in S)

def sum_pchr(M, S):
    return sum(qhat(M.delete(M.groundset() - set(F))) * \
            kl(M.contract(F)) for F in S)

def set_division(M, e):
    L = M.lattice_of_flats()
    divs = {'A': set(), 'B': set(), 'C': set(), 'D': set()}
    for x in L[1:]:
        if set(e).issubset(set(x)):
            if frozenset(set(x) - set(e)) in L:
                divs['A'].add(x)
            else:
                divs['D'].add(x)
        else:
            if frozenset(set(x) | set(e)) in L:
                divs['C'].add(x)
            else:
                divs['B'].add(x)
    return divs

def prhc_at_flats(M, L):
    return {F: str(kl(M.delete(M.groundset() - set(F))) * qhat(M.contract(F))) for F in L}

In [33]:
def leading_term(p):
    return p.leading_coefficient() * t ** p.degree()

def lt(M):
    r = M.rank()
    if r % 2 == 1:
        return leading_term(kl_inverse_fast(M))
    else:
        return 0

def circuits_containing_element(M, e):
    circuits_with_e = [circuit for circuit in M.circuits() if e.issubset(set(circuit))]
    return circuits_with_e

def sum_at_Circuits(M, C):
    sum = 0
    for c in C:
        sum += t * kl_inverse_fast(M.contract(c)) *\
        lt(M.delete(M.groundset() - c).contract(e))
    return sum

In [34]:
import networkx as nx
import ipycytoscape

def interactive_graph(M):
    L = M.lattice_of_flats()
    G = nx.DiGraph()

    rank_colors = ['#FF0000', '#00FF00', '#FFFF00', '#FF00FF']
    prhc_vals = prhc_at_flats(M, list(L))

    flat_to_label = {flat: f"{i}: {prhc_vals.get(flat, 0)}" for i, flat in enumerate(L)}
    for flat, label in flat_to_label.items():
        G.add_node(label, rank=M.rank(flat), color=rank_colors[M.rank(flat) % len(rank_colors)])

    for x, y in L.cover_relations():
        G.add_edge(flat_to_label[y], flat_to_label[x])

    cyto_graph = ipycytoscape.CytoscapeWidget()
    cyto_graph.graph.add_graph_from_networkx(G)

    style = [
        {
            'selector': 'node',
            'style': {
                'label': 'data(id)',
                'background-color': 'data(color)',
                'text-valign': 'center',
                'text-halign': 'center',
                'color': 'black',
                'font-size': '10px',
                'width': 20,
                'height': 20
            }
        },
        {
            'selector': 'edge',
            'style': {
                'curve-style': 'straight',
                'width': 1,
                'line-color': '#999',
                'target-arrow-color': '#0000',
                'target-arrow-shape': 'none'
            }
        }
    ]
    cyto_graph.set_style(style)

    pos = nx.multipartite_layout(G, subset_key='rank', align='horizontal', scale=1.5)
    for node in cyto_graph.graph.nodes:
        node.position = {'x': pos[node.data['id']][0] * 50 * M.rank(), 'y': -pos[node.data['id']][1] * 100 * M.rank()}

    cyto_graph.set_layout(name='preset')
    return cyto_graph


In [35]:
m, n = 3, 3

G, e = parallel_connection(m, n)
M = Matroid(G)

interactive_graph(M)

CytoscapeWidget(cytoscape_layout={'name': 'preset'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'data(id)', 'background-color': 'data(color)', 'text-valign': 'center', 'text-halign': 'center', 'color': 'black', 'font-size': '10px', 'width': 20, 'height': 20}}, {'selector': 'edge', 'style': {'curve-style': 'straight', 'width': 1, 'line-color': '#999', 'target-arrow-color': '#0000', 'target-arrow-shape': 'none'}}], graph=Graph(edges=[Edge(classes=' directed ', data={'source': '1: 2', 'target': '0: -t - 4'}), Edge(classes=' directed ', data={'source': '2: 2', 'target': '0: -t - 4'}), Edge(classes=' directed ', data={'source': '3: -1', 'target': '1: 2'}), Edge(classes=' directed ', data={'source': '3: -1', 'target': '2: 2'}), Edge(classes=' directed ', data={'source': '4: 2', 'target': '0: -t - 4'}), Edge(classes=' directed ', data={'source': '5: -1', 'target': '2: 2'}), Edge(classes=' directed ', data={'source': '5: -1', 'target': '4: 2'}), Edge(classes=' directed ', data={'

In [36]:
from prettytable import PrettyTable

table = PrettyTable()
table.field_names = ["n", "QMn", "QMdel", "Qdiff", "Qcon", "diffQcon", "extra term"]

m = 5
for n in range(3, 15):
    G, e = parallel_connection(m, n)
    M = Matroid(G)
    QMn = S(qhat(Matroid(G)))
    QMdel = S(uniformQhat(n+m-3, n+m-2))
    Qdiff = QMn - QMdel
    Qcon = S(uniformQhat(n-2, n-1) * uniformQhat(m-2, m-1))
    diffQcon = Qdiff + ((-1)^M.rank() * t + 1 ) * Qcon
    extra_term = sum_at_Circuits(M, circuits_containing_element(M, e))
    table.add_row([n, QMn, QMdel, Qdiff, Qcon, diffQcon, extra_term])

table.max_width = 25  # Set the maximum width for each column

print(table)

+----+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+
| n  |            QMn            |           QMdel           |           Qdiff           |            Qcon           |          diffQcon         |         extra term        |
+----+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+---------------------------+
| 3  |     -3*t^2 - 11*t - 8     |      -5*t^2 - 9*t - 5     |      2*t^2 - 2*t - 3      |          2*t + 3          |            -3*t           |        4*t^2 + 3*t        |
| 4  |     14*t^2 + 24*t + 12    |     14*t^2 + 14*t + 6     |          10*t + 6         |          -4*t - 6         |           -4*t^2          |           4*t^2           |
| 5  | -10*t^3 - 32*t^2 - 41*t - | -14*t^3 - 28*t^2 - 20*t - |  4*t^3 - 4*t^2 - 21*t - 9 |      4*t^2 + 12*t + 9     |       